# 6) List of historical overlapping districts

For every SMD past and present, generate a list of SMDs that overlapped with it.

In other words, turn the overlap CSVs (one row for every overlap) into a list for every district.

In [1]:
import pandas as pd

In [2]:
districts = pd.read_csv('../data/districts.csv')
overlap_forwards = pd.read_csv('overlap_forwards.csv')
overlap_backwards = pd.read_csv('overlap_backwards.csv')

In [3]:
overlap_forwards['valid'] = True
overlap_backwards['valid'] = True

In [4]:
# The overlap area must by at least 10 meters square
overlap_forwards.loc[overlap_forwards.overlap_area < 10, 'valid'] = False
overlap_backwards.loc[overlap_backwards.overlap_area < 10, 'valid'] = False

In [5]:
# Manually remove some overlaps
# overlap_forwards = 
overlap_forwards.loc[
    (overlap_forwards.smd_id_2012 == 'smd_6B06') & (overlap_forwards.smd_id_2022 == 'smd_2022_8F01')
    , 'valid'
] = False

overlap_backwards.loc[
    (overlap_backwards.smd_id_2012 == 'smd_6B06') & (overlap_backwards.smd_id_2022 == 'smd_2022_8F01')
    , 'valid'
] = False

In [6]:
# Limit to only the valid overlaps
overlap_forwards = overlap_forwards[overlap_forwards.valid].copy()
overlap_backwards = overlap_backwards[overlap_backwards.valid].copy()

In [7]:
of_list = overlap_forwards.groupby('smd_id_2022').smd_id_2012.apply(lambda x: ', '.join(sorted(list(x))))

In [8]:
ob_list = overlap_backwards.groupby('smd_id_2012').smd_id_2022.apply(lambda x: ', '.join(sorted(list(x))))

In [9]:
df = pd.DataFrame(pd.concat([of_list, ob_list]), columns=['overlap_smds'])
df.index.name = 'smd_id'
df = df.reset_index()

In [10]:
df['redistricting_cycle'] = df.smd_id.apply(lambda x: 2022 if '_2022_' in x else 2012)

In [11]:
# Copy this to OpenANC Source
df.sort_values(by=['redistricting_cycle', 'smd_id']).overlap_smds.to_clipboard(index=False)